# Process an interferogram with ASF HyP3

https://hyp3-docs.asf.alaska.edu/using/sdk/ 

## Search for scenes

scenes over grand mesa, colorado using https://asf.alaska.edu/api/

In [ ]:
import requests
import shapely.geometry

roi = shapely.geometry.box(-108.3,39.2,-107.8,38.8)
polygonWKT = roi.wkt

baseurl = "https://api.daac.asf.alaska.edu/services/search/param"

data = dict(
    intersectsWith=polygonWKT,
    platform='Sentinel-1',
    processingLevel="SLC",
    beamMode='IW',
    output='json',
    start='2020-10-30T11:59:59Z',
    end='2020-11-30T11:59:59Z',
    #relativeOrbit=None,
    #flightDirection=None,
)

r = requests.get(baseurl, params=data, timeout=100)
print(r.url)

In [ ]:
# load results into pandas dataframe
import pandas as pd
df = pd.DataFrame(r.json()[0])
df.head()

In [ ]:
# Easier to explore the inventory in plots
import hvplot.pandas
from bokeh.models.formatters import DatetimeTickFormatter

formatter = DatetimeTickFormatter(years='%m-%d')
timeseries = df.hvplot.scatter(x='startTime', y='relativeOrbit', c='relativeOrbit',
                               xformatter=formatter,
                               title='Acquisition times (UTC)')

In [ ]:
import geopandas as gpd
import geoviews as gv
import panel as pn

gf_aoi = gpd.GeoDataFrame(geometry=[roi])
polygons = df.stringFootprint.apply(shapely.wkt.loads)
gf_footprints = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=polygons)

tiles = gv.tile_sources.StamenTerrainRetina.options(width=600, height=400)
aoi = gf_aoi.hvplot(geo=True, fill_color=None, line_color='m', hover=False)
footprints = gf_footprints.hvplot.polygons(geo=True, legend=False, alpha=0.2, c='relativeOrbit', title='Sentinel-1 Tracks') 

mapview = tiles * footprints * aoi

pn.Column(mapview,timeseries)

In [ ]:
df.relativeOrbit.unique()

In [ ]:
orbit = '129'
reference = '2020-11-11'
secondary = '2020-10-30'

dfS = df[df.relativeOrbit == orbit]
granule1 = dfS.loc[dfS.sceneDate.str.startswith(reference), 'granuleName'].values[0]
granule2 = dfS.loc[dfS.sceneDate.str.startswith(secondary), 'granuleName'].values[0]
print(f'granule1: {granule1}')
print(f'granule2: {granule2}')

In [ ]:
for ref in [reference, secondary]:
    print(dfS.loc[dfS.sceneDate.str.startswith(ref), 'downloadUrl'].values[0])

## Process an InSAR pair (interferogram)

examples:
- https://nbviewer.jupyter.org/github/ASFHyP3/hyp3-sdk/blob/main/docs/sdk_example.ipynb
- https://hyp3-docs.asf.alaska.edu/using/sdk/

In [ ]:
import hyp3_sdk

In [ ]:
# ~/.netrc file used for credentials
hyp3 = hyp3_sdk.HyP3()

In [ ]:
# Processing quota
hyp3.check_quota() #199 (200 scenes per month?)

In [ ]:
job = hyp3.submit_insar_job(granule1,
                            granule2,
                            name='gm_20201111_20201030', 
                            include_los_displacement=True, 
                            include_inc_map=True)

In [ ]:
# All jobs you've submitted
# NOTE: processing w/ defaults uses INSAR_GAMMA 
# NOTE: re-run this cell to update results of batch job
batch = hyp3.find_jobs()
job = batch.jobs[0] # most recent job
job

In [ ]:
# If you have lists of dictionaries, visualizing with a pandas dataframe is convenient
df = pd.DataFrame([job.to_dict() for job in batch])
df.head()

In [ ]:
# Actually no, expiration time is not available for download...
#pd.to_datetime(df.expiration_time[0]) - pd.to_datetime(df.request_time[0])

In [ ]:
# ImportError: IProgress not found. Please update jupyter and ipywidgets.
# but I think this still succeeeds
job.download_files()

In [ ]:
!ls -ltrh

In [ ]:
# requires ipywidgets
#hyp3.watch(job)

## Process multiple pairs in batch mode

In [ ]:
# with progress bar
#from tqdm.auto import tqdm 

#insar_jobs = sdk.Batch()
#for reference in tqdm(granules):
#    neighbors_metadata = asf_search.get_nearest_neighbors(reference, max_neighbors=2)
#    for secondary_metadata in neighbors_metadata:
#        insar_jobs += hyp3.submit_insar_job(reference, secondary_metadata['granuleName'], name='insar-example')
#print(insar_jobs)

In [ ]:
# Can also submit jobs via web interface # Can also visit https://hyp3.asf.alaska.edu/pending_products 
# Which then shows logs that can be sorted into 'submitted, failed, etc...'